In [2]:
# 用来保持连接的
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


# 1. 安装环境

In [3]:
# 克隆的仓库在./GPT2-Chinese 目录下
!git clone https://github.com/Morizeyao/GPT2-Chinese.git

Cloning into 'GPT2-Chinese'...
remote: Enumerating objects: 280, done.
remote: Total 280 (delta 0), reused 0 (delta 0), pack-reused 280
Receiving objects: 100% (280/280), 13.44 MiB | 6.50 MiB/s, done.
Resolving deltas: 100% (140/140), done.


In [4]:
# 安装需求文件
!python3 -m pip install -r ./GPT2-Chinese/requirements.txt

     |████████████████████████████████| 311 kB 29.9 MB/s 
     |████████████████████████████████| 5.8 MB 54.5 MB/s 
     |████████████████████████████████| 52.9 MB 46 kB/s 
     |████████████████████████████████| 1.2 MB 63.8 MB/s 
     |████████████████████████████████| 880 kB 68.2 MB/s 
     |████████████████████████████████| 132 kB 73.3 MB/s 
     |████████████████████████████████| 79 kB 10.4 MB/s 
     |████████████████████████████████| 8.8 MB 57.7 MB/s 
     |████████████████████████████████| 127 kB 80.8 MB/s 
  Created wheel for thulac: filename=thulac-0.2.1-py3-none-any.whl size=53141671 sha256=6aba296513449f49eaaf37fb82dd041c1e72e1760c924eed0bd8e5b9b9f1901b
  Stored in directory: /root/.cache/pip/wheels/97/37/f3/be4ae10faf0fbf35cc192469b737ead6f8f99404bcd82fb2e0
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=379c5c14ecfb0e713544e7e5c8473e51a2b5e0b29b8ab8ce6a378963bcb98b85
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83e

In [5]:
# 移动到GPT2-Chinese 路径
%cd ./GPT2-Chinese

/content/GPT2-Chinese


In [6]:
!mkdir pretrained
%cd ./pretrained

/content/GPT2-Chinese/pretrained


**下载pretrained model 到新创建的pretrained文件夹**

用的是通用中文模型（小）

想要用别的模型 到这个repo 找gdrive 链接 https://github.com/Morizeyao/GPT2-Chinese

然后拷贝到自己的gdrive就可以，右上三个点-> add shortcut ->然后在快捷方式里对应的文件选make copy

再将copy下来的文件改为谁都有修改权限，将分享链接/d/后面的id复制下来就可以下载了

In [7]:
!gdown --id 16FmH5ZjpejKLNUtmdTfteZlREQkuVyl5
!gdown --id 13Trje1N7YU_1U9iGLpMjRrIFZlg5HC1-
!ls

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=16FmH5ZjpejKLNUtmdTfteZlREQkuVyl5
To: /content/GPT2-Chinese/pretrained/pytorch_model.bin
100% 421M/421M [00:04<00:00, 92.8MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=13Trje1N7YU_1U9iGLpMjRrIFZlg5HC1-
To: /content/GPT2-Chinese/pretrained/config.json
100% 605/605 [00:00<00:00, 1.21MB/s]
config.json  pytorch_model.bin


In [8]:
%cd ..

/content/GPT2-Chinese


# 2. 转换训练用文本
先将你的训练txt发到colab file，以train.txt命名, 记得放在GPT2-Chinese文件夹下

In [9]:
# 删掉原本的json
!rm -f train.json
!mkdir data

In [10]:
# 根据你的训练文本创建train.json

"""
Author: godweiyang
"""

import json

dic = {}
with open("train.txt", "r", encoding="utf8") as f:
    merge_line = ""
    for line in f:
        line = line.strip()
        merge_line += line
        if len(merge_line) > 500:
            dic[merge_line] = 1
            merge_line = ""

with open("./data/train.json", "w", encoding="utf8") as f:
    json.dump(dic, f, ensure_ascii=False)

# 3. 训练

In [11]:
#@title
# 参数用途
!python train.py -h

2022-05-20 23:47:28.062920: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
usage: train.py [-h] [--device DEVICE] [--model_config MODEL_CONFIG]
                [--tokenizer_path TOKENIZER_PATH]
                [--raw_data_path RAW_DATA_PATH]
                [--tokenized_data_path TOKENIZED_DATA_PATH] [--raw]
                [--epochs EPOCHS] [--batch_size BATCH_SIZE] [--lr LR]
                [--warmup_steps WARMUP_STEPS] [--log_step LOG_STEP]
                [--stride STRIDE]
                [--gradient_accumulation GRADIENT_ACCUMULATION] [--fp16]
                [--fp16_opt_level FP16_OPT_LEVEL]
                [--max_grad_norm MAX_GRAD_NORM] [--num_pieces NUM_PIECES]
                [--min_length MIN_LENGTH] [--output_dir OUTPUT_DIR]
                [--pretrained_model PRETRAINED_MODEL]
                [--writer_dir WRITER_DIR] [--segment] [-

In [ ]:
# 超过memory就把batch_size改小点
!python train.py --raw --epochs 1 --batch_size 2 --num_pieces 1 --min_length 3 --pretrained_model pretrained

2022-05-20 23:47:33.688990: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
args:
Namespace(batch_size=2, bpe_token=False, device='0,1,2,3', encoder_json='tokenizations/encoder.json', epochs=1, fp16=False, fp16_opt_level='O1', gradient_accumulation=1, log_step=1, lr=0.00015, max_grad_norm=1.0, min_length=3, model_config='config/model_config_small.json', num_pieces=1, output_dir='model/', pretrained_model='pretrained', raw=True, raw_data_path='data/train.json', segment=False, stride=768, tokenized_data_path='data/tokenized/', tokenizer_path='cache/vocab_small.txt', vocab_bpe='tokenizations/vocab.bpe', warmup_steps=2000, writer_dir='tensorboard_summary/')
config:
{
  "attn_pdrop": 0.1,
  "embd_pdrop": 0.1,
  "finetuning_task": null,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_la

# 4. 生成文本
一定要等训练完不然会报错

原repo不能断点续传

如果想再已经训练的模型基础上继续训练可以用`./model`下面存储的model替换pretrained里的model

In [ ]:
# length决定长度， nsamples 后面决定生成多少个例子。
input = "半年时间，眨眼便过。"
!python3 generate.py --length=500 --nsamples=3 --prefix=$input --fast_pattern